In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일 읽어오기

PINECONE_API_KEY=os.environ['PINECONE_API_KEY']

## index 생성

In [16]:
from pinecone import Pinecone
# 클라이언트 생성
pinecone_client=Pinecone(api_key=PINECONE_API_KEY)

In [17]:
pinecone_client

In [21]:
from pinecone import ServerlessSpec
pinecone_client.create_index(
    name='embedding-3d',
    dimension=3,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "embedding-3d",
    "metric": "cosine",
    "host": "embedding-3d-6aohn7n.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 3,
    "deletion_protection": "disabled",
    "tags": null
}

In [23]:
index=pinecone_client.Index('embedding-3d')
index

In [24]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [25]:
for ids in index.list(namespace='embedding-3d-ns1'):
    print(ids)

In [26]:
index.upsert(  
  vectors=[{ 'id' : 'vec1' , 'values' :[1.0, 1.5, 2.0] , 'metadata' :   {'genre':'drama'}},  
    { 'id' : 'vec2' , 'values' :[2.0, 1.5, 0.5] , 'metadata' : {'genre':'action'}},  
    { 'id' : 'vec3' , 'values' :[0.1, 0.3, 0.5] , 'metadata' : {'genre':'drama'}},  
    { 'id' : 'vec4' , 'values' :[1.0, 2.5, 3.5] , 'metadata' : {'genre':'action'}},  
    { 'id' : 'vec5' , 'values' :[3.0, 1.2, 1.3] , 'metadata' : {'genre':'action'}},  
    { 'id' : 'vec6' , 'values' :[0.3, 1.1, 2.5] , 'metadata' : {'genre':'drama'}},  
    ],  
  namespace='embedding-3d-ns1'  
  ) #update + insert  

{'upserted_count': 6}

### id list

In [28]:
for ids in index.list(namespace='embedding-3d-ns1'):
    print(ids)

['vec1', 'vec2', 'vec3', 'vec4', 'vec5', 'vec6']


## query

In [ ]:
response=index.query(
    namespace='embedding-3d-ns1',
    vector=[0.1,0.4,0.7],
    top_k=3,
    include_values=True,
    include_metadata=True,
    filter={'genre':{'$eq':'drama'}}
    
)
response

{'matches': [{'id': 'vec3',
              'metadata': {'genre': 'drama'},
              'score': 1.00008953,
              'values': [0.1, 0.3, 0.5]},
             {'id': 'vec6',
              'metadata': {'genre': 'drama'},
              'score': 0.99550879,
              'values': [0.3, 1.1, 2.5]},
             {'id': 'vec4',
              'metadata': {'genre': 'action'},
              'score': 0.990372956,
              'values': [1.0, 2.5, 3.5]}],
 'namespace': 'embedding-3d-ns1',
 'usage': {'read_units': 1}}